In [7]:
%matplotlib notebook
#Import pandas and numpy to handle data
import pandas as pd
import numpy as np

#import libraries for accessing the database
import psycopg2
from sqlalchemy import create_engine
from postgres_credentials import *

#import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import libraries for tokenization and ML
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer;


#Import all libraries for creating a deep neural network
#Sequential is the standard type of neural network with stackable layers
from keras.models import Sequential;
#Dense: Standard layers with every node connected, dropout: avoids overfitting
from keras.layers import Dense, Dropout, Activation;

In [8]:
#Querying the database
def query_database(tabletweets):
    engine = create_engine("postgresql+psycopg2://%s:%s@%s:%d/%s" %(usertwitter, passwordtwitter, hosttwitter, porttwitter, dbnametwitter))
    table = pd.read_sql_query('select * from %s' %tabletweets,con=engine, index_col='id')
    return table

In [ ]:
#preprocess text in tweets by removing links, @UserNames, blank spaces, etc.
def preprocessing_text():
    
    

In [9]:
#Processing the data: Tokenization
def tokenization_tweets(table):
    tokenization = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')
    tokenization.fit_on_texts(table['tweet'])
    return tokenization

In [11]:
#split dataset
dictionary = tokenization_tweets(query_database('tweets_avengers'))
a = dictionary.word_index

In [ ]:
#Visualizing the data

In [33]:
#Step2: Create a Neural Network

In [5]:
#Create the model
model_nn = Sequential()

In [ ]:
if __name__ == "__main__":
    
    